# Формирование SQL запросов для базы данных EdTech-проекта 

## 1. Исходные данные

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Дана таблица ***peas***:

| **Название атрибута**  | **Тип атрибута**  | **Смысловое значение**  |
|:-:|:-:|:--|
|  pea_id | str  | id "горошины" |
|  st_id | int  | id ученика |
| timest | timestamp  | Время решения задания |
| correct | bool | Правильно ли решено задание? |
| subject | text | Дисциплина, к которой относится задание  |

Таблица ***studs***:

| **Название атрибута**  | **Тип атрибута**  | **Смысловое значение**  |
|:-:|:-:|:--|
|  st_id | int  | id ученика |
| test_grp | text  | Метка ученика в данном эксперименте |


Таблица ***checks***:

| **Название атрибута**  | **Тип атрибута**  | **Смысловое значение**  |
|:-:|:-:|:--|
|  st_id | int  | id ученика |
| sale_time | timestamp  | Время покупки |
| money | int | Цена, по которой приобрели данный курс |
| subject | text | Дисциплина, на которую приобрели полный доступ  |

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 3 "горошины" в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты. Метка ученика в эксперименте отражена в таблице ***studs***.


## 2. Задачи

**Задача 1.** Студенты, которые хотя бы раз за текущий месяц правильно решили 5 горошин за час, классифицируются как усердные. Необходимо написать запрос, который даст информацию о количестве усердных студентов за март 2020 года.

**Задача 2.** Необходимо в одном запросе выгрузить следующую информацию о группах пользователей (тестовой и контрольной):
- ARPU 
- ARPAU 
- CR в покупку 
- СR активного пользователя в покупку 
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

## 3. Механика решения

Данные, на которых можно было бы выполнять поэтапное построение SQL-запросов и проверять их корректность, отсутствуют. Поэтому первым шагом выполнения задания является генерация исходных данных. 

После того, как данные, соответствующие условиям предметной области, будут сгенерированы, к сформированным таблицам будут производиться запросы на SQL. Для преобразования из SQL в код Python будет использоваться библиотека ***pandasql***. Эта библиотека использует движок SQLite, поэтому запросы будут написаны под него.

## Генерация исходных данных

Все данные генерируются для одного года - 2020-го.

При генерации используются следующие предположения:

* Для таблицы **checks**:
    * Цена приобретения курса - фиксирована в течение года, скидок не предусмотрено;
    * Вероятность покупки одной дисциплины для каждого студента - 0,3;
    * Количество дисциплин, которые куплены студентом - биномиально распределённая случайная величина на отрезке от 0 до 5; 
    * Дата и время покупки - равномерно распределённая случайная величина на отрезке от 1 до длительности года в секундах. Реализация случайной величины для конкретного студента и предмета будет означать время покупки подписки на этот предмет этим студентом в секундах от начала года, приведённое к формату даты и времени;

* Для таблицы **studs**:
    * Вероятность попадания ученика в тестовую группу равна 0,5;


* Для таблицы **peas**:
    * На каждую дисциплину приходится одинаковое количество "горошин";
    * Каждая "горошина" может быть пройдена только один раз, либо с положительным, либо с отрицательным результатом;
    * "Горошины" независимы друг от друга, каждая может быть пройдена независимо от того, пройдены ли другие;
    * Вероятность правильного решения рассчитывается через логит-преобразование нормально распределённой случайной величины со средним 0,7 и стандартным отклонением 1, которое округляется до целого (0 или 1). 0 означает, что "горошина" не решена, 1 - "решена". Параметр 0,7 примерно соответствует вероятности верного решения;
    * Дата и время решения задания определяется в два этапа
        - Сначала для всех студентов на основе двух сгенерированных случайных величин: 
            - равномерно распределённой случайной величины от 1 до 365, которая определяет день, когда студент решает задания;
            - равномерно распределённой случайной величины от 1 до длительности дня в секундах, которая определяет время решения студентом горошины. 
        - Затем для части студентов, которых предполагается считать "усердными", случайным образом для каждого студента выбирается день и час в году, когда он будет иметь повышенную интенсивность решения "горошин". Количество решённых заданий (независимо от правильности решения) для каждого студента в этот период подчинено нормальному распределению с параметрами: средняя = 10, ст. откл. = 3. Чтобы время решения не было одинаковым, выбранный день и час корректируется на количество секунд, подчинённое нормальному распределению с центром в нуле и стандартным отклонением, равным 1800 (30 минут * 60 секунд). 
    * Поскольку при генерации решений усердными студентами не учитывается ограничение на максимальное число "горошин" в день для тех, кто не имеет оплаченной подписки, далее вносятся соответствующие корректировки, удаляющие результаты прохождения "горошин", не соответствующие данным ограничениям. 

Количественные ограничения: 
* количество студентов - 500. 
* количество "горошин" - 100 для каждой дисциплины. 
* количество дисциплин - 5: базы данных, статистика, программирование, визуализация, математика.

In [1]:
import random
from math import exp
import pandas as pd
import numpy as np
from datetime import timedelta as td
from datetime import datetime as dt
from pandasql import sqldf

from tqdm.auto import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
# исходные данные для генерации таблиц

students = 500
subjects = ['databases', 'statistics', 'programming', 'dataviz', 'math']
prices = [2000, 5000, 3000, 4000, 3000]

# приближённая вероятность верного ответа в одной "горошине"
correct_probability = 0.7

# число горошин в одном предмете
peas_num = 100 

# вероятность попадания в тестовую группу
test_group_probability = 0.5

# вероятность покупки подписки
buy_probability = 0.3

# ограничение на число "горошин" в день
peas_limit = 3

# модельная доля усердных студентов
active_students_prop = 0.3

# модельные параметры распределения числа попыток для усердных студентов
hard_mean, hard_sigma = 10, 3


# функция определения группы
def groupfunc(x):
    if x == 1:
        return 'test'
    else:
        return 'control'

In [73]:
# создание таблицы studs
st_id = np.arange(1, students + 1)
test_grp = np.random.binomial(1, test_group_probability, students)
studs = pd.DataFrame({'st_id': st_id, 
                      'test_grp': test_grp})

studs.test_grp = studs.test_grp.apply(groupfunc)

# создание таблицы checks
count_paying_courses = np.random.binomial(len(subjects), buy_probability, students)
paying_courses = [random.sample(subjects, i) for i in count_paying_courses]

st_id_list = []
subjects_list = []
j = 0

for i in paying_courses:
    j += 1
    st_id_list.extend([j] * len(i))
    subjects_list.extend(i)

checks = pd.DataFrame({'st_id': st_id_list, 
                       'subject': subjects_list})
checks['money'] = checks.subject.apply(lambda x: prices[subjects.index(x)])
day_in_second = 60 * 60 * 24
year_in_second = 60 * 60 * 24 * 365

sec_payments = [int(np.random.uniform(1, year_in_second)) for i in np.arange(1, len(checks) + 1)]
payment_dates = [str(pd.to_datetime("01.01.2020", format="%d.%m.%Y") + td(seconds=i)) for i in sec_payments]
checks['sale_time'] = payment_dates

In [74]:
# создание таблицы peas
'''
Сначала таблица peas создаётся для всех студентов и предметов, т.е. общее число "горошин"
в ней равно числу студентов, умноженному на число предметов и число "горошин" в предмете.
Время прохождения горошины распределено равномерно в течение года. 
'''

peas_ids = []
for i in subjects:
    for j in np.arange(1, int(1 + peas_num)):
        for k in np.arange(1, students + 1):
            peas_ids.append(i + "_" + str(j) + "_" + str(k))

peas_dates = []
z = np.random.normal(correct_probability, 1, len(peas_ids))

start_date = pd.to_datetime("01.01.2020", format="%d.%m.%Y")
day_peas = [int(np.random.uniform(1, 365)) for i in np.arange(1, len(peas_ids) + 1)]
sec_peas = [int(np.random.uniform(1, day_in_second)) for i in np.arange(1, len(peas_ids) + 1)]

for j in tqdm(range(len(day_peas))):
        pea_date = start_date + td(days=day_peas[j]) + td(seconds=sec_peas[j])
        peas_dates.append(str(pea_date))


peas = pd.DataFrame({'pea_id': peas_ids,
                     'timest': peas_dates,
                     'correct': z})
peas['st_id'] = list(st_id) * peas_num * len(subjects)
peas['subject'] = peas.pea_id.apply(lambda x: x.split("_")[0])
peas['correct'] = peas.correct.apply(lambda x: int(np.round(exp(x) / (1 + exp(x)))))

'''
Далее необходимо создать группу усердных студентов, которые решают "горошины" с повышенной интенсивностью.
На основе заданной доли случайным образом выбираются такие студенты. После чего для выбранных студентов
случайным образом выбирается тот день года и период времени, когда они будут активны и число горошин, которое они решат
в этот период. 
'''

active_studs_ids = studs.st_id.sample(n = int(active_students_prop * students))

for i in tqdm(active_studs_ids):
    peas_count = int(np.random.normal(hard_mean, hard_sigma)) 
    try:
        peas_active = peas[peas.st_id == i].sample(n = peas_count)

        rand_day = np.random.randint(365)
        rand_hour = np.random.randint(24)

        for j in np.arange(1,len(peas_active)+1):
            rand_sec = np.random.normal(0, 30*60, size = peas_count+1).astype(int)
            peas_active['timest'] = [str(pd.to_datetime("01.01.2020", format="%d.%m.%Y") + \
                                         td(days = rand_day, hours = rand_hour, seconds = int(rand_sec[i]))) \
                                         for i in np.arange(1, peas_count+1)]
            
            pea_active_ids = peas_active.pea_id.sort_index().index.tolist()
            peas.loc[pea_active_ids, 'timest'] = peas_active['timest'].sort_index()
            
    except ValueError:
        pass
    
'''
Не все выбранные пользователи могут иметь платную подписку, и выполнить требуемое условие "усердия", 
поэтому скорректируем полученные данные, удалив отметки о решениях, которые выходят за пределы 5 "горошин" в день
у таких студентов. Для упрощения будем считать число "горошин" в день, а не в сутки, т.е. в рамках дня учитывать 
только те "горошины", которые решены в период с 00:00 до 23:59 этого дня.
'''

# агрегируем данные о числе решённых "горошин" по дням, предметам и студентам, фильтруем те, где имеется превышение
peas_dates = peas
peas_dates['timest'] = pd.to_datetime(peas_dates.timest)
peas_dates['timest_date'] = peas_dates.timest.dt.normalize()
peas_count_by_days = peas.groupby(['st_id','subject','timest_date'], as_index = False) \
                         .agg({'pea_id':'count'}) \
                         .query('pea_id > @peas_limit')

# выбираем тех студентов и предметы, где подписка не оплачивалась, т.е. превышения быть не должно
peas_count_sales = peas_count_by_days.merge(checks, on = ['st_id', 'subject'], how = 'left')
peas_count_sales['limit_exceed'] = np.where( \
    (peas_count_sales.sale_time > peas_count_sales.timest_date) | \
                                            (peas_count_sales.sale_time.isnull() == True), 1, 0)
peas_count_sales['peas_overlimit'] = np.where(peas_count_sales.limit_exceed == 1, \
                                              peas_count_sales.pea_id - peas_limit, 0)

# создаём таблицу со сведениями о количестве "горошин" в разрезе студентов и предметов, которые превышают лимит
overlimit = peas_count_sales[peas_count_sales.peas_overlimit != 0][['st_id', 'subject', 'peas_overlimit']]

# добавляем в таблицу "горошин" сведения о превышении лимита и находит те id "горошин", 
# которые решены в превышении лимита
peas_cut = peas.merge(overlimit, on = ['st_id', 'subject'], how = 'inner')

peas_cut["rank"] = peas_cut.groupby(['st_id', 'subject', 'timest_date'])['timest'] \
                           .rank(method="first", ascending=True)
peas_cutted = peas_cut[peas_cut['rank'] > peas_limit].pea_id

# удаляем из исходной таблицы горошин те, которые превышают лимит
peas = peas[~np.isin(peas.pea_id,peas_cutted)].drop('timest_date', axis = 1)

'''
Поскольку мы предполагаем, что большая часть студентов не будет пробовать проходить все горошины, то 
случайным образом исключим часть записей из этой таблицы.
'''

drop_indices = np.random.choice(peas.index, int(0.3 * len(peas.index)), replace=False)
peas = peas.drop(drop_indices)

  0%|          | 0/250000 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

In [134]:
# финальные данные

display(checks.head())
display(studs.head())
display(peas.head())

,st_id,subject,money,sale_time
0,1,programming,3000,2020-04-09 23:28:44
1,2,programming,3000,2020-03-07 19:05:23
2,2,databases,2000,2020-06-16 16:00:14
3,2,math,3000,2020-04-04 19:45:10
4,2,dataviz,4000,2020-03-21 03:55:03


,st_id,test_grp
0,1,test
1,2,test
2,3,test
3,4,control
4,5,control


,pea_id,timest,correct,st_id,subject
0,databases_1_1,2020-09-02 04:04:51,1,1,databases
1,databases_1_2,2020-01-20 17:28:46,0,2,databases
3,databases_1_4,2020-09-27 19:36:44,1,4,databases
5,databases_1_6,2020-10-09 20:34:52,1,6,databases
6,databases_1_7,2020-07-18 12:28:51,0,7,databases


## Решение задачи 1

Ученики, которые хотя бы раз за текущий месяц правильно решили 5 горошин за час и больше, классифицируются как усердные. Необходимо написать запрос, который даст информацию о количестве усердных студентов за март 2020 года.

In [85]:
# Запрос, который рассматривает выполнение условия только в интервале с начала до конца часа

q = '''
    WITH hard_students AS 
        (
            SELECT 
                SUBSTR(timest, 0, 14) as date_hour,
                st_id,
                subject,
                SUM(correct) as sum_correct 
            FROM peas
            WHERE correct = 1
                AND SUBSTR(timest, 0, 8) = '2020-03'
            GROUP BY st_id, subject, date_hour
            HAVING sum_correct >= 5
            ORDER BY sum_correct DESC
        )
        
    SELECT COUNT(*) as hard_students_count FROM hard_students;
    '''

In [86]:
%%timeit
sqldf(q)
# время выполнения запроса

2.02 s ± 137 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [76]:
# результат
sqldf(q)

,hard_students_count
0,1


## Альтернативное решение задачи 1

Если рассматривать выполнение условия независимо от начала и конца часа, то для получения числа усердных студентов необходимо написать запрос с использованием оконной функции.  

In [77]:
# Запрос, который рассматривает выполнение условия в любом часовом интервале.
# К сожалению, SQLite не поддерживает в RANGE интервал в формате даты, поэтому результат запроса некорректный.
# Аналогичный запрос (кроме части c RANGE BETWEEN) можно написать для PostgreSQL или Clickhouse, где реализована
# поддержка интервала дат в окне. 

q = '''
    WITH correct_peas AS 
        (
            SELECT 
                timest,
                st_id,
                subject,
                correct
            FROM peas
            WHERE correct = 1
                AND SUBSTR(timest, 0, 8) = '2020-03'
            ORDER BY st_id, subject, timest
        ),
        sum_correct_peas AS
        (
            SELECT 
                datetime(timest),
                st_id,
                subject,
                SUM(correct) OVER w AS sum_correct
            FROM correct_peas
            WINDOW w AS 
                (
                PARTITION BY st_id, subject
                ORDER BY datetime(timest) ASC
                RANGE BETWEEN 3600 PRECEDING AND CURRENT ROW
                )
        )
        
    SELECT COUNT(st_id) AS hard_students_count
    FROM sum_correct_peas
    WHERE sum_correct >= 5
    '''

In [78]:
# результат
sqldf(q)

,hard_students_count
0,0


## Решение задачи 2

**Задача 2.** Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:
- ARPU 
- ARPAU 
- CR в покупку 
- СR активного пользователя в покупку 
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

Поскольку все данные нужно получить и для тестовой, и для контрольной группы, то итоговая таблица должна иметь размерность 2х5. Для решения задачи рассчитывались промежуточные таблицы с использованием обобщённого табличного выражения. На основе этих таблиц формируется расчётная таблица, которая является основой для формирования итоговой. 

**Активными в контексте решения данной задачи считаются те же студенты, что и при решении задачи 1.**

In [87]:
# pandasql использует SQLite, который не поддерживает создание переменных
# для определения критерия усердного ученика используется часть HAVING sum_correct >= 5

q = '''
    WITH hard_students AS 
        (
            SELECT 
                SUBSTR(timest, 0, 14) as date_hour,
                st_id,
                subject,
                SUM(correct) as sum_correct 
            FROM peas
            WHERE correct = 1
            GROUP BY st_id, subject, date_hour
            HAVING sum_correct >= 5
        ),
        
        rev_by_users AS
        (
            SELECT 
                test_grp, 
                st_id,
                SUM(money) as active_users_rev
            FROM hard_students
                INNER JOIN studs USING (st_id)
                LEFT JOIN checks USING (st_id)
            GROUP BY test_grp, st_id
        ),
        
        rev_by_groups AS
        (
            SELECT test_grp,
                COUNT(active_users_rev) AS active_paying_users,
                COUNT(st_id) AS active_users,
                SUM (active_users_rev) AS active_users_rev
            FROM rev_by_users
            GROUP BY test_grp
        ),
        
        math_tab AS 
        (
            SELECT 
                test_grp, 
                SUM(money) AS math_rev,
                COUNT (DISTINCT studs.st_id) AS math_paying_users
            FROM checks
                INNER JOIN studs USING (st_id)
            WHERE subject = 'math'
            GROUP BY test_grp
        ),
        
        active_users AS 
        (
            SELECT 
                test_grp,           
                COUNT (DISTINCT checks.st_id) AS paying_users,
                COUNT (DISTINCT studs.st_id) AS users,
                SUM (money) AS total_rev,
                math_paying_users
            FROM studs
                LEFT JOIN checks USING (st_id)
                INNER JOIN math_tab USING (test_grp)
            GROUP BY test_grp
        ),
        
        calc_table AS
        (
        SELECT 
            test_grp,
            paying_users,
            users,
            total_rev,
            active_users,
            active_paying_users,
            active_users_rev,
            math_paying_users
        FROM active_users
        INNER JOIN rev_by_groups 
            USING (test_grp)
        )
        
    SELECT 
        test_grp AS grp,
        ROUND(total_rev / paying_users, 0) AS ARPU,
        ROUND(active_users_rev / active_paying_users, 0) AS ARPAU,
        ROUND(CAST(paying_users AS REAL) / CAST(users AS REAL),2) AS CR,
        ROUND(CAST(active_paying_users AS REAL) / CAST(active_users AS REAL),2) AS CR_active,
        ROUND(CAST(math_paying_users AS REAL) / CAST(users AS REAL),2) AS CR_math
    FROM calc_table
    '''

In [81]:
%%timeit
sqldf(q)
# время выполнения запроса

2.3 s ± 151 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [88]:
sqldf(q) \
    .rename(index={0: 'control_group', 1: 'test_group'}) \
    .drop(columns = 'grp')

,ARPU,ARPAU,CR,CR_active,CR_math
control_group,6073.0,5666.0,0.83,0.6,0.28
test_group,6170.0,3400.0,0.84,1.0,0.28


## Заключение

Введённые при генерации данных предположения позволяют получить таблицы, соответствующие указанным в условии и приближенным к реальному проекту. При этом критерий "усердности" является для сгенерированных данных достаточно жёстким, т.к. ему удовлетворяет малое число студентов. 